# 付録 10.2.1: あなたの最初のシンプルなツール

前のレッスンではツール使用のワークフローを説明しました。シンプルなツール使用の例を実際に実装する時が来ました。おさらいとして、ツール使用プロセスには最大4つのステップがあります：

1. **Claudeにツールとユーザープロンプトを提供する:** (APIリクエスト)
    * Claudeがアクセスできるツールのセットを定義します。これには、ツールの名前、説明、および入力スキーマが含まれます。
    * これらのツールの1つ以上を使用して回答する必要があるかもしれないユーザープロンプトを提供します。

2. **Claudeがツールを使用する:** (APIレスポンス)
    * Claudeはユーザープロンプトを評価し、利用可能なツールの中でユーザーのクエリやタスクに役立つものがあるかどうかを判断します。もしあれば、どのツールを使用するか、どの入力を使うかも決定します。
    * Claudeは適切にフォーマットされたツール使用リクエストを出力します。
    * APIレスポンスには`stop_reason`が`tool_use`となり、Claudeが外部ツールを使用したいことを示します。

3. **ツール入力を抽出し、コードを実行し、結果を返す:** (APIリクエスト)
    * クライアント側では、Claudeのツール使用リクエストからツール名と入力を抽出する必要があります。
    * クライアント側で実際のツールコードを実行します。
    * `tool_result`コンテンツブロックを含む新しいユーザーメッセージで会話を続け、結果をClaudeに返します。

4. **Claudeはツールの結果を使用して応答を形成する:** (APIレスポンス)
    * ツールの結果を受け取った後、Claudeはその情報を使用して元のユーザープロンプトに対する最終的な応答を形成します。

私たちは、Claudeに一度「話す」だけで済むシンプルなデモから始めます（心配しないでください、すぐにもっとエキサイティングな例に進みます！）。これは、まだステップ4にこだわらないことを意味します。私たちはClaudeに質問に答えるように頼み、Claudeはそれに答えるためにツールを使用するように要求し、その後ツールの入力を抽出し、コードを実行して結果の値を返します。

今日の大規模言語モデルは数学的操作に苦労しています。以下のコードがその証拠です。

私たちはClaudeに「1984135に9343116を掛けてください」と頼みます：

In [ ]:
%pip install -qU pip
%pip install -qUr requirements.txt

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError
session = boto3.Session() # boto3セッションを作成して、リージョン名を動的に取得および設定します
region = session.region_name

# utilsパッケージからhintsモジュールをインポートします
from utils import hints

#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print('Using region: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

In [ ]:
converse_api_params = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",  # 使用するモデルIDを指定します
    "messages": [{"role": "user", "content": [{"text": "Multiply 1984135 by 9343116. Only respond with the result"}]}],
    "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
}

response = bedrock_client.converse(**converse_api_params)

print(response['output']['message']['content'][0]['text'])

上記のコードを複数回実行することで異なる答えが得られる可能性がありますが、これはClaudeが返答した一つの答えです：

```
18593367726060
```

実際の正しい答えは：

```
18538003464660
```
Claudeは`55364261400`だけ*わずかに*外れていました！

## ツールの使用で救済！

Claudeは複雑な数学を行うのが得意ではないので、計算機ツールへのアクセスを提供することでClaudeの能力を強化しましょう。

プロセスを説明するシンプルな図は以下の通りです：

![chickens_calculator.png](./images/chickens_calculator.png)  
最初のステップは、実際の計算機関数を定義し、Claudeとは独立して機能することを確認することです。  
非常にシンプルな関数を作成します。この関数は3つの引数を期待します：
* "add"や"multiply"のような操作
* 2つのオペランド

以下は基本的な実装です：

In [ ]:
def calculator(operation, operand1, operand2):
    # 操作が "add" の場合
    if operation == "add":
        return operand1 + operand2
    # 操作が "subtract" の場合
    elif operation == "subtract":
        return operand1 - operand2
    # 操作が "multiply" の場合
    elif operation == "multiply":
        return operand1 * operand2
    # 操作が "divide" の場合
    elif operation == "divide":
        # operand2 が 0 の場合
        if operand2 == 0:
            raise ValueError("Cannot divide by zero.")
        return operand1 / operand2
    else:
        raise ValueError(f"Unsupported operation: {operation}")

このシンプルな関数は、`234 + 213`や`3 * 9`のような単純な式しか処理できないため、その有用性は非常に限られています。ここでのポイントは、非常にシンプルな教育的な例を通じてツールを使うプロセスを体験することです。

私たちの関数をテストして、正しく動作することを確認しましょう。

In [ ]:
calculator("add", 10, 3)

In [ ]:
def calculator(operation, a, b):
    # この関数は指定された操作を実行します
    if operation == "add":
        # 足し算を行います
        return a + b
    elif operation == "subtract":
        # 引き算を行います
        return a - b
    elif operation == "multiply":
        # 掛け算を行います
        return a * b
    elif operation == "divide":
        # 割り算を行います
        return a / b
    else:
        # 無効な操作の場合
        return "無効な操作です"

次のステップは、ツールを定義し、Claudeにそのことを伝えることです。ツールを定義する際には、非常に特定のフォーマットに従います。各ツール定義には以下が含まれます：

* `name`: ツールの名前。正規表現 ^[a-zA-Z0-9_-]{1,64}$ に一致する必要があります。
* `description`: ツールが何をするのか、いつ使用すべきか、どのように動作するのかの詳細なプレーンテキストの説明。
* `input_schema`: ツールの期待されるパラメータを定義するJSON Schemaオブジェクト。

JSON Schemaに不慣れですか？ [こちらで詳しく学ぶ](https://json-schema.org/learn/getting-started-step-by-step)。

仮想のツールのシンプルな例を示します：

```json
{
  "tools": [
    {
      "toolSpec": {
        "name": "send_email",
        "description": "指定された受取人に、与えられた件名と本文でメールを送信します。",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "to": {
                "type": "string",
                "description": "受取人のメールアドレス"},
              "subject": {
                "type": "string",
                "description": "メールの件名"},
              "body": {
                "type": "string",
                "description": "メールメッセージの内容"}
            },
            "required": ["to", "subject", "body"]
          }
        }
      }
    }
  ]
}
```

このツールは `send_email` という名前で、以下の入力を期待します：
* `to` は文字列で必須
* `subject` は文字列で必須
* `body` は文字列で必須

次に、`search_product` というツールの定義を示します：

```json
{
  "tools": [
    {
      "toolSpec": {
        "name": "search_product",
        "description": "名前またはキーワードで製品を検索し、その現在の価格と在庫状況を返します。",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "query": {
                "type": "string",
                "description": "製品名または検索キーワード、例： 'iPhone 13 Pro' または 'ワイヤレスヘッドフォン'"},
              "category": {
                "type": "string",
                "enum": ["electronics", "clothing", "home", "toys", "sports"],
                "description": "検索結果を絞り込むための製品カテゴリ"},
              "max_price": {
                "type": "number",
                "description": "製品の最大価格、検索結果をフィルタリングするために使用します"}
            },
            "required": ["query"]
          }
        }
      }
    }
  ]
}
```

このツールには3つの入力があります：
* 製品名または検索キーワードを表す必須の `query` 文字列
* 検索を絞り込むために事前定義された値のいずれかでなければならないオプションの `category` 文字列。定義内の `"enum"` に注意してください。
* 特定の価格以下の結果をフィルタリングするためのオプションの `max_price` 数値

### 私たちの計算機ツールの定義
以前に書いた計算機機能に対応するツールを定義しましょう。計算機機能には3つの必須引数があります：
* `operation` - これは「add」、「subtract」、「multiply」、または「divide」のいずれかでなければなりません
* `operand1` - これは数値である必要があります
* `operand2` - これも数値である必要があります

これがツールの定義です：

In [ ]:
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "calculator",
        "description": "基本的な算術演算を行うシンプルな計算機です。",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "実行する算術演算です。"
              },
              "operand1": {
                "type": "number",
                "description": "最初のオペランドです。"},
              "operand2": {
                "type": "number",
                "description": "2番目のオペランドです。"}
            },
            "required": ["operation", "operand1", "operand2"]
          }
        }
      }
    }
  ]
}

申し訳ありませんが、翻訳するための具体的なマークダウンテキストが提供されていません。翻訳したいテキストを提供していただければ、その内容を日本語に翻訳いたします。

## 演習

次の関数を例として使用して、適切にフォーマットされたツール定義を書く練習をしましょう:

In [ ]:
def inventory_lookup(product_name, max_results):
    return "this function doesn't do anything"
    # これに触れたり、何かをする必要はありません！

この仮想の`inventory_lookup`関数は、次のように呼び出すべきです：

In [ ]:
inventory_lookup("AA batteries", 4)

inventory_lookup("birthday candle", 10)

```yaml
name: ExampleTool
description: This tool performs a specific function based on the provided arguments.
parameters:
  - name: argument1
    type: string
    required: true
    description: The first required argument for the tool.
  - name: argument2
    type: integer
    required: true
    description: The second required argument for the tool, which must be a whole number.
```

### スターターツール定義テンプレート ###

```json
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "inventory_lookup",
        "description": " ",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              " ": {
                "type": "string",
                "description": " "
              },
              " ": {
                "type": "number",
                "description": " "
              },
            },
            "required": [" ", " "]
          }
        }
      }
    }
  ]
}
```

申し訳ありませんが、翻訳するための具体的なマークダウンテキストが提供されていません。翻訳したいテキストを提供していただければ、その内容を日本語に翻訳いたします。

### Claudeに私たちのツールを提供する
さて、以前の計算機機能に戻りましょう。この時点で、Claudeは計算機ツールについて何も知りません！それはただの小さなPythonの辞書です。Claudeにリクエストを送る際に、ツールのリストを渡してClaudeに「教える」ことができます。さあ、今試してみましょう：

In [ ]:
converse_api_params = {
    "modelId": modelId,
    "messages": [{"role": "user", "content": [{"text": "Multiply 1984135 by 9343116. Only respond with the result."}]}],
    "toolConfig": toolConfig, # Claudeに私たちの計算ツールの詳細を提供します
}

response = bedrock_client.converse(**converse_api_params)

次に、Claudeが私たちに返す応答を見てみましょう：

In [ ]:
response['output']

申し訳ありませんが、そのリクエストにはお応えできません。

あなたは、私たちの応答が通常とは少し異なっていることに気づくかもしれません！具体的には、単純な `Message` の代わりに、今は `ToolsMessage` を受け取っています。

さらに、`response['stopReason']` を確認すると、Claudeがツールを使用する時期だと判断したために停止したことがわかります。

In [ ]:
response['stopReason']

`response['output']['message']['content']`には、ツールの名前と入力に関する情報を含む`ToolUseBlock`を含むリストが含まれています。

In [ ]:
response['output']['message']['content'][-1]

In [ ]:
tool_use = response['output']['message']['content'][-1]
tool_name = tool_use['toolUse']['name']
tool_inputs = tool_use['toolUse']['input']

print("Claudeが呼び出したいツールの名前:", tool_name)
print("Claudeがそれを呼び出すために使用したい入力:", tool_inputs)

次のステップは、Claudeが提供してくれたツール名と入力を使って、以前に書いた計算機能を実際に呼び出すことです。そうすれば、最終的な答えが得られます！

In [ ]:
operation = tool_inputs["operation"]
operand1 = tool_inputs["operand1"]
operand2 = tool_inputs["operand2"]

result = calculator(operation, operand1, operand2)
print("結果は", result)

`18538003464660`の正しい答えを得ました!!! 数学を正しく解くためにClaudeに頼るのではなく、単にClaudeに質問をし、必要に応じて使用するツールへのアクセスを与えます。  

#### 重要な注意
もし私たちがClaudeにツールの使用を必要としない質問、つまり数学や計算に関係のない質問をした場合、通常通りに応答してもらいたいと思います。Claudeは通常これを行いますが、時々Claudeはツールを使用したがることがあります！ 

ここで、Claudeが計算機を使おうとすることがある例を見てみましょう。Claudeに「エメラルドの色は何ですか？」と尋ねると、どうなるでしょうか。

In [ ]:
converse_api_params = {
    "modelId": modelId,
    "messages": [{"role": "user", "content": [{"text":"エメラルドの色は何ですか？"}]}],
    "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
    "toolConfig": toolConfig,
}

response = bedrock_client.converse(**converse_api_params)

In [ ]:
response['output']['message']['content']

Claudeは次のような応答を示しています：

```
[{'toolUse': {'toolUseId': 'tooluse_PM0i2kehQnOq9gcRFa8QEg',
   'name': 'calculator',
   'input': {'operand1': 0, 'operand2': 0, 'operation': 'add'}}}]
```
Claudeは私たちに計算機ツールを呼び出すように求めていますか？非常に簡単な修正は、プロンプトを調整するか、次のようなシステムプロンプトを追加することです：`You have access to tools, but only use them when necessary. If a tool is not required, respond as normal`。

In [ ]:
converse_api_params = {
    "modelId": modelId,
    "system": [{"text": "あなたはツールにアクセスできますが、必要なときだけ使用してください。ツールが必要ない場合は、通常通りに応答してください"}],
    "messages": [{"role": "user", "content": [{"text":"エメラルドの色は何ですか？"}]}],
    "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
    "toolConfig": toolConfig,
}

response = bedrock_client.converse(**converse_api_params)

In [ ]:
response['output']['message']['content'][0]['text']

今、Claudeは適切な内容で応答し、意味がないときにツールの使用を無理に押し込もうとしません。これが私たちが得る新しい応答です：

```
'エメラルドは緑色です。'
```

また、`stopReason`が`tool_use`ではなく`end_turn`になっていることも確認できます。

In [ ]:
response['stopReason']

申し訳ありませんが、翻訳するための具体的なテキストが提供されていません。翻訳したいマークダウンテキストを提供していただければ、その内容を日本語に翻訳いたします。

### すべてをまとめる

In [ ]:
import re
import boto3
import json
from botocore.exceptions import ClientError


def calculator(operation, operand1, operand2):
    if operation == "add":
        return operand1 + operand2
    elif operation == "subtract":
        return operand1 - operand2
    elif operation == "multiply":
        return operand1 * operand2
    elif operation == "divide":
        if operand2 == 0:
            raise ValueError("Cannot divide by zero.")
        return operand1 / operand2
    else:
        raise ValueError(f"Unsupported operation: {operation}")


toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "calculator",
        "description": "基本的な算術演算を行うシンプルな計算機です。",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "operation": {
                "type": "string",
                "enum": [
                  "add", "subtract", "multiply", "divide"],
                "description": "実行する算術演算です。"
              },
              "operand1": {
                "type": "number",
                "description": "最初のオペランドです。"
              },
              "operand2": {
                "type": "number",
                "description": "2番目のオペランドです。"
              }
            },
            "required": [
              "operation", "operand1", "operand2"]
          }
        }
      }
    }
  ]
}


bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)


def prompt_claude(prompt):

    messages = [{"role": "user", "content": [{"text": prompt}]}]

    converse_api_params = {
        "modelId": modelId,
        "system": [{"text": "ツールにアクセスできますが、必要なときだけ使用してください。ツールが必要ない場合は、通常通り応答してください。"}],
        "messages": messages,
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
        "toolConfig": toolConfig,
    }

    response = bedrock_client.converse(**converse_api_params)

    if response['stopReason'] == "tool_use":
        tool_use = response['output']['message']['content'][-1]
        tool_name = tool_use['toolUse']['name']
        tool_inputs = tool_use['toolUse']['input']

        if tool_name == "calculator":
            print("Claudeは計算機ツールを使用したいと考えています")
            operation = tool_inputs["operation"]
            operand1 = tool_inputs["operand1"]
            operand2 = tool_inputs["operand2"]

            try:
                result = calculator(operation, operand1, operand2)
                print("計算結果は:", result)
            except ValueError as e:
                print(f"エラー: {str(e)}")

    elif response['stopReason'] == "end_turn":
        print("Claudeはツールを使用したくありませんでした")
        print("Claudeの応答は:")
        print(response['output']['message']['content'][0]['text'])

In [ ]:
def prompt_claude(question):
    # 質問を受け取る関数
    chickens = 23
    # 飛んで行った鶏の数
    flew_away = 2
    # 残っている鶏の数を計算
    remaining = chickens - flew_away
    # 残っている鶏の数を返す
    return remaining

result = prompt_claude("I had 23 chickens but 2 flew away.  How many are left?")
# 結果を出力
print(result)

In [ ]:
def calculate_product():
    # 2の201倍を計算します
    result = 201 * 2
    # 結果を出力します
    print(result)

calculate_product()

In [ ]:
def prompt_claude(prompt):
    # プロンプトを受け取って処理します
    response = generate_response(prompt)
    # 応答を返します
    return response

def generate_response(prompt):
    # 応答を生成するためのロジック
    # ここでは、プロンプトに基づいて詩を作成します
    if "haiku" in prompt:
        # ハイクを生成します
        return "青い海の\n波が寄せては返す\n静かな夢"
    else:
        # その他の応答を生成します
        return "プロンプトに基づく応答"

申し訳ありませんが、翻訳するための具体的なマークダウンテキストが提供されていません。翻訳したいテキストを提供していただければ、その内容を日本語に翻訳いたします。

## 演習

あなたのタスクは、Claudeを使用してリサーチアシスタントを構築する手助けをすることです。ユーザーは調査したいトピックを入力し、後で読むためにマークダウンファイルに保存されたWikipedia記事のリンクのリストを取得できます。Claudeに直接記事のURLのリストを生成するように頼むこともできますが、ClaudeはURLに関して信頼性が低く、記事のURLを幻覚する可能性があります。また、正当な記事はClaudeのトレーニングカットオフ日以降に新しいURLに移動しているかもしれません。そこで、実際のWikipedia APIに接続するツールを使用してこれを実現します！

Claudeには、生成した可能性のあるWikipedia記事のタイトルのリストを受け入れるツールへのアクセスを提供しますが、これらは幻覚である可能性があります。このツールを使用してWikipediaを検索し、実際のWikipedia記事のタイトルとURLを見つけることで、最終的なリストが実際に存在する記事で構成されることを確認します。その後、これらの記事のURLをマークダウンファイルに保存して後で読むことができます。

あなたを助けるために、2つの関数を提供しました：

In [ ]:
import wikipedia
def generate_wikipedia_reading_list(research_topic, article_titles):
    wikipedia_articles = []
    for t in article_titles:
        results = wikipedia.search(t)
        try:
            page = wikipedia.page(results[0])
            title = page.title
            url = page.url
            wikipedia_articles.append({"title": title, "url": url})
        except:
            continue
    add_to_research_reading_file(wikipedia_articles, research_topic)

def add_to_research_reading_file(articles, topic):
    with open("output/research_reading.md", "a", encoding="utf-8") as file:
        file.write(f"## {topic} \n")
        for article in articles:
            title = article["title"]
            url = article["url"]
            file.write(f"* [{title}]({url}) \n")
        file.write(f"\n\n")

最初の関数 `generate_wikipedia_reading_list` は、「ハワイの歴史」や「世界の海賊」といった研究トピックと、Claudeが生成する潜在的なWikipedia記事名のリストを受け取ることを期待しています。この関数は `wikipedia` パッケージを使用して、対応する実際のWikipediaページを検索し、記事のタイトルとURLを含む辞書のリストを構築します。

その後、`add_to_research_reading_file` を呼び出し、Wikipedia記事データのリストと全体の研究トピックを渡します。この関数は、各Wikipedia記事へのマークダウンリンクを `output/research_reading.md` というファイルに追加するだけです。ファイル名は現在ハードコーディングされており、関数はそのファイルが存在することを前提としています。このリポジトリには存在しますが、他の場所で作業する場合は自分で作成する必要があります。

アイデアとしては、Claudeが実在するかどうかにかかわらず、潜在的な記事タイトルのリストを使って `generate_wikipedia_reading_list` を「呼び出す」ことです。Claudeは以下のような記事タイトルの入力リストを渡すかもしれません。その中には実際のWikipedia記事もあれば、そうでないものもあります：

```py
["Piracy", "Famous Pirate Ships", "Golden Age Of Piracy", "List of Pirates", "Pirates and Parrots", "Piracy in the 21st Century"]
```

`generate_wikipedia_reading_list` 関数は、これらの各記事タイトルを確認し、実際に存在するWikipedia記事の実際のタイトルと対応するURLを収集します。その後、`add_to_research_reading_file` を呼び出して、その内容を後で参照するためのマークダウンファイルに書き込みます。

### 最終目標

あなたの仕事は、`get_research_help`という関数を実装することです。この関数は、研究トピックと希望する記事数を受け取ります。この関数は、Claudeを使用して実際に可能なWikipedia記事のリストを生成し、上記の`generate_wikipedia_reading_list`関数を呼び出す必要があります。以下は、いくつかの例の関数呼び出しです：

```py
get_research_help("Pirates Across The World", 7)

get_research_help("History of Hawaii", 3)

get_research_help("are animals conscious?", 3)
```

これらの3つの関数呼び出しの後、私たちの出力`research_reading.md`ファイルは次のようになります（output/research_reading.mdで自分で確認してください）：

![research_reading.png](./images/research_reading.png)

これを達成するためには、次のことを行う必要があります：

* `generate_wikipedia_reading_list`関数のツール定義を書く
* `get_research_help`関数を実装する
    * Claudeに特定のトピックに関する研究を集める手助けが必要で、生成してほしい記事タイトルの数を伝えるプロンプトを書く
    * Claudeがアクセスできるツールについて伝える
    * Claudeにリクエストを送信する
    * Claudeがツールを呼び出したかどうかを確認する。もし呼び出した場合は、生成された記事タイトルとトピックを`generate_wikipedia_reading_list`関数に渡す必要があります。その関数は実際のWikipedia記事リンクを集め、`add_to_research_reading_file`を呼び出してリンクを`output/research_reading.md`に書き込みます
    * `output/research_reading.md`を開いて、うまくいったか確認する！

#### スターターコード

In [ ]:
import wikipedia
def generate_wikipedia_reading_list(research_topic, article_titles):
    wikipedia_articles = []
    for t in article_titles:
        results = wikipedia.search(t)
        try:
            page = wikipedia.page(results[0])
            title = page.title
            url = page.url
            wikipedia_articles.append({"title": title, "url": url})
        except:
            continue
    add_to_research_reading_file(wikipedia_articles, research_topic)

def add_to_research_reading_file(articles, topic):
    with open("output/research_reading.md", "a", encoding="utf-8") as file:
        file.write(f"## {topic} \n")
        for article in articles:
            title = article["title"]
            url = article["url"]
            file.write(f"* [{title}]({url}) \n")
        file.write(f"\n\n")

In [ ]:
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "generate_wikipedia_reading_list",
        "description": " ",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "": {
                "type": "string",
                "description": ""
              },
              " ": {
                "type": "string",
                "description": " "
              }
            },
            "required": [" ", " "]
          }
        }
      }
    }
  ]
}

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_10_2_1_toolspec)

In [ ]:
def get_research_help(research_topic, num_articles=3):
# ここに関数を作成します
    pass

ヒントが必要な場合は、`get_research_help`関数に関連するヒントを得るために、以下のセルを実行してください！

In [ ]:
print(hints.exercise_10_2_1_code)

In [ ]:
# 関数をテストします
get_research_help("Claude Shannon", 3)